In [84]:
import os
import numpy as np
from skimage.draw import polygon
import cv2
import json
from shutil import copyfile
from pathlib import Path
from skimage.measure import find_contours
import glob
import base64
import skimage


In [98]:
def getFileNameInfor(filepath):
    file_name = filepath.split('/')[-1]
    file_base = Path(filepath).stem
    fdir = os.path.dirname(os.path.abspath(filepath))
    return (fdir, file_name, file_base)

In [99]:
def boundary2Mask(xs, ys, img):
    mask = np.zeros(img.shape[0:2])
    rr, cc = polygon(xs, ys, img.shape)
    mask[rr, cc] = 255
    return mask


In [108]:
def mask2Boundary(mask_file, sample_ratio=5):
    mask = cv2.imread(mask_file)[:,:,0] 
    contours = find_contours(mask, 200)
    boundaries=[]
    for i, verts in enumerate(contours):
        new_vects = []
        for k, p in enumerate(verts):
            # Subtract the padding and flip (y, x) to (x, y)z
            if k % sample_ratio==0:
                new_vects.append(list(p))
        boundaries.append(new_vects)
    return boundaries

masks = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/proj_outdir/1456/masks/0_yin.jpg'
print(mask2Boundary(masks))

[[[907.2156862745098, 493.0], [907.2142857142857, 488.0], [907.2125984251968, 483.0], [906.2156862745098, 479.0], [905.0, 475.7843137254902], [902.0, 473.7843137254902], [899.0, 471.78], [896.0, 469.78740157480314], [892.0, 468.7843137254902], [887.0, 468.7843137254902], [882.0, 468.7843137254902], [878.0, 469.7843137254902], [875.7843137254902, 472.0], [875.7843137254902, 477.0], [876.7843137254902, 481.0], [878.7843137254902, 484.0], [880.7874015748032, 487.0], [883.0, 489.2173913043478], [885.7826086956521, 492.0], [888.7843137254902, 494.0], [892.0, 495.21259842519686], [896.0, 496.2156862745098], [901.0, 496.2165354330709], [904.2125984251968, 495.0], [907.2156862745098, 493.0]]]


In [109]:
def masks2json(mask_file_ls, img_file, out_dir):
    img_dir, img_name, img_id = getFileNameInfor(img_file)
    json_file="{}/{}.json".format(out_dir,image_id)
    # image = skimage.io.imread(img_file)
    image = cv2.imread(img_file) 
    imageHeight = image.shape[0]    # 获得长宽
    imageWidth = image.shape[1]
    with open(img_file, 'rb') as f:   # 编码图片 base64
        imageData = base64.b64encode(f.read())
        imageData = str(imageData, encoding='utf-8')
    json_data = {
        'version': "4.5.6",
        "flags": {},
        'shapes': [],
        'imagePath': img_file,
        'imageData': imageData,
        'imageHeight': imageHeight,
        'imageWidth': imageWidth
    }
    
    for i,m in enumerate(mask_file_ls):
        m_dir, m_name, m_id = getFileNameInfor(m)
        className = m_id.split("_")[-1]
        boundary = mask2Boundary(m)[0]
        obj = {
            'ID': i,
            'label': className,
            'points': [],
            'group_id': None,
            'shape_type': "polygon",
            'flags': {},
        }
        for point in boundary:
            obj["points"].append([float(point[0]), float(point[1])])
        json_data['shapes'].append(obj)
    json_str = json.dumps(json_data, indent=4)
    with open('{}/{}.json'.format(out_dir, img_id), 'w') as json_file:
        json_file.write(json_str)
    copyfile(img_file, '{}/{}.png'.format(out_dir, img_id))

# test
img_file = "/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/proj_outdir/1456/1456.png"
img_dir, img_name, img_id = getFileNameInfor(img_file) 
mask_dir= "{}/masks/".format(img_dir)
mask_file_ls = glob.glob('{}/*.jpg'.format(mask_dir))
out_dir = "/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/out_json"
masks2json(mask_file_ls, img_file, out_dir)

In [ ]:
import sys
ROOT_DIR = os.path.abspath("../../")  # 指定根目录
sys.path.append(ROOT_DIR)  # 查找库的本地版本
from mrcnn import utils

def removeDuplicateObject(r, min_ratio=0.7):
    classid_ls = list(map(int, r['class_ids']))
    scores = r["scores"]
    masks = r["masks"]

    new_classid_ls = []
    new_scores = []
    new_masks = None
    removed_indexes = []
    for i in range(len(classid_ls)-1):
        for k in range(i+1, len(classid_ls)):
            if k in removed_indexes:
                continue
            mask_a = np.reshape(masks[:,:,i],(masks.shape[0],masks.shape[1],1))
            class_a = classid_ls[i]
            mask_b = np.reshape(masks[:,:,k],(masks.shape[0],masks.shape[1],1))
            class_b = classid_ls[k]
            #print(utils.compute_overlaps_masks(mask_a, mask_b))
            #print('as')
            if utils.compute_overlaps_masks(mask_a, mask_b)[0]>min_ratio:
                if class_a<class_b:
                    removed_indexes.append(k)
                else:
                    removed_indexes.append(i)
                continue
        if i in removed_indexes:
            continue
        else:
            new_classid_ls.append(classid_ls[i])
            new_scores.append(scores[i])
            if new_masks is None:
                new_masks=np.reshape(masks[:,:, i],(masks.shape[0],masks.shape[1],1))
            else:
                mask=np.reshape(masks[:,:, i],(masks.shape[0],masks.shape[1],1))
                new_masks=np.concatenate([new_masks, mask], axis=-1)
    return {'class_ids':new_classid_ls, "scores":new_scores, "masks":new_masks}

In [ ]:
def predictToJson(image_path, model, out_dir):
    img_dir, img_name, img_id = getFileNameInfor(img_file)
    image = skimage.io.imread(image_path)
    imageHeight = image.shape[0]    # 获得长宽
    imageWidth = image.shape[1]
    rr = model.detect([image], verbose=1)[0]
    r = removeDuplicateObject(rr,0.6)
    with open(image_path, 'rb') as f:   # 编码图片 base64
        imageData = base64.b64encode(f.read())
        imageData = str(imageData, encoding='utf-8')
    json_data = {
        'version': "4.5.6",
        "flags": {},
        'shapes': [],
        'imagePath': img_dir,
        'imageData': imageData,
        'imageHeight': imageHeight,
        'imageWidth': imageWidth
    }
    for i in range(0, len(r['class_ids'])):
        class_id = int(r['class_ids'][i])
        className = ['BG', 'yin', 'yin-yang', 'yang']   # 把数字对应上标签
        score = int(r["scores"][i])
        mask = r["masks"][:, :, i]
        boundary = mask2Boundary(mask)
        obj = {
            'ID': i,
            'label': className[class_id],
            'points': [],
            'group_id': None,
            'shape_type': "polygon",
            'flags': {},
        }
        i = 0
        for point in boundary:
            obj["points"].append([float(point[0]), float(point[1])])
        json_data['shapes'].append(obj)
    json_str = json.dumps(json_data, indent=4)
    with open('{}/{}.json'.format(out_dir, img_id), 'w') as json_file:
        json_file.write(json_str)
    copyfile(img_file, '{}/{}.png'.format(out_dir, img_id))

In [66]:

from skimage import data
a = data.camera()
b = data.camera()*2
masks = np.concatenate([a, b], axis=-1)

print(masks[:,:,0])

k=np.array([1,2])
p=np.array([3,4])
#np.append(k, p)
#a[:,:,1]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [110]:

def json2masks(json_file, img_file, out_dir):
    img_dir, img_name, image_id = getFileNameInfor(img_file)
    img = cv2.imread(img_file) 
    with open(json_file) as JSON:
        data = json.load(JSON)
        image_name = data['imagePath']
        image_height = data['imageHeight']
        image_width = data['imageWidth']
        shapes = data["shapes"]
        os.makedirs('{}/{}'.format(out_dir,image_id), exist_ok=True)
        os.makedirs('{}/{}/masks'.format(out_dir,image_id), exist_ok=True)
        copyfile(img_file, '{}/{}/{}'.format(out_dir,image_id,img_name))
        for i, s in enumerate(shapes):
            label = s['label']
            points = s['points']
            xs = []
            ys = []
            for p in points:
                ys.append(p[0])
                xs.append(p[1])
            mask = boundary2Mask(xs, ys, img)
            cv2.imwrite("{}/{}/masks/{}_{}.jpg".format(out_dir, image_id,i,label), mask)
           

In [112]:
def json2cell_infor(json_file):
    with open(json_file) as JSON:
        data = json.load(JSON)
        image_name = data['imagePath']
        shapes = data["shapes"]
        cell_dic = {}
        for i, s in enumerate(shapes):
            label = s['label']
            if label not in cell_dic:
                cell_dic[label]=0
            cell_dic[label]+=1
        return cell_dic
            
            
    

In [114]:
test_img = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/datasource/1456.png'
image_id = '1456'
test_json = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/datasource/1459.json'
out_dir = '/home/hqyone/mnt/2tb/github/MaskRCNN_TF1/samples/hpv_models/test/data/proj_outdir'
# 
# json2masks(test_json,test_img, out_dir)
print(json2cell_infor(test_json))




{'yin': 24, 'yin-yang': 3, 'yang': 1}
